# Using Deep Transfer Learning to detect COVID-19

Can we use Deep Learning to analyze a set of chest XRays of known COVID-19 patients to detect when a patient has COVID-19.

The work and information in this notebook is inspired by a [blog post](https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/) by Adrian Rosebrock on his website [PyImageSearch](www.pyimagesearch.com)

In that blog post, Adrian shows how to apply transfer learning using the [VGG16 Convolution Neural Network](https://neurohive.io/en/popular-networks/vgg16/) using the network weights from the ImageNet dataset.  

My contribution to that great blog post is to go into some detail on what transfer learning is and why it was so powerful in this application.  I will then show how we can use other Deep Learning Convolutional networks for transfer learning and their performance on the same dataset.

Lastly we will look at how a model trained on the COVID-19 XRay images does against images of normal chest XRays and chest XRays of those with Pneumonia to see if the model can accurately detect that chest XRay is NOT COVID-19.


<font color="red" size="5">Like Adrian, I want to be very clear that the information here, and derived from [Adrian's](https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/) blog post is meant for Deep Learning educational purposes only.  In no way is it implied that this technique should be used for the identification of COVID-19.</font>

## DataSets

### COVID-19 Image Dataset

There is a Github repo started by Joseph Paul Cohen called [covid-chestxray-dataset](https://github.com/ieee8023/covid-chestxray-dataset).  

![Covid19Share](./notebook_images/covid19-share-image.png)

From the README.md file in that Github repo:

`We are building a database of COVID-19 cases with chest X-ray or CT images. We are looking for COVID-19 cases as well as MERS, SARS, and ARDS.`

We will use this dataset to have a model learn what a chest XRay looks like that has COVID-19.

This dataset is updated frequently.  New images were added as I was working with the dataset so check back often.

### Kaggle Chest X-Ray Images (Pneumonia)

[Kaggle](www.kaggle.com) is an online community of people interested in data science.  It allows users to find, publish, explore and build models around datasets made available to the public.  

The dataset we will use is the [Chest X-Ray Images (Pneumonia)](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia) dataset.  This dataset has chest X-Ray images of normal lungs as well as chest X-Ray images of lungs with Pneumonia.

We can use this dataset to train the model on what normal chest X-Rays look like.

### Visual Inspect of Datasets

<br>
<table>
  <tr style="text-align:center; background-color:white">
    <td> <img src="./notebook_images/covid19-collage.png" alt="COVID" style="width: 400px;"/></td>
    <td> <img src="./notebook_images/normal-collage.png" alt="NORMAL" style="width: 400px;"/> </td>
    <td> <img src="./notebook_images/pneumonia-collage.png" alt="PNEU" style="width: 400px;"/> </td>
  </tr>
  <tr style="width:100%;">
    <td style="width:20%;"> <p style="font-family:overpass;font-size:16px;text-align:center;color:#303030;font-weight:300;">COVID-19</p> </td>
    <td style="width:20%;"> <p style="font-family:overpass;font-size:16px;text-align:center;color:#303030;font-weight:300;">NORMAL</p> </td>
    <td style="width:20%;"> <p style="font-family:overpass;font-size:16px;text-align:center;color:#303030;font-weight:300;">PNEUMONIA</p> </td>

  </tr>
</table>

<br>


### Caveats

I would like to stress that the datasets being used are not vetted by myself or as far as I know, anyone with expertise in the field.  We are using datasets from disparate sources, collected at different times with different procedures.  I have no way of knowing if the image is really of a COVID-19 Chest X-Ray, or some other ailament that resembles COVID-19.

So take this exercise as an interesting use case of applying Deep Transfer Learning to a set of images for classification.  

## What is Deep Transfer Learning

Convolutional Neural Networks (CNNs) are commonly used when working with images and Deep Learning.

Think of a convolution as a small, 3x3 or 5x5, etc, mathematical matrix that is applied to an image to alter the image.  We use convolutions all time when using image processing software to sharpen or blur an image.  

A CNN model will be made up of some number of convolutional layers.  Each layer will have a different number of kernels ( small matrix we talked about ), and a final fully connected network (FCN) layer that will be used to perform the actual classification step.

The initial convolutional layers, also called the convolutional base, act as a feature extraction layer.  This layer is attempting to identify the features in a dataset and for an image that might be interesting parts of the images. According to Francois Chollet the creator of Keras from his book Deep Learning with Python,

`... the representations learned by the convolutional base as likely to be more generic [than the fully connected layer] and therefore more reusable; the feature maps of the convnet are presence maps of generic concepts over a picture, which is likely to be useful regardless of the conputer-vision problem at hand.`

This means that the convolution layers can be trained to identify interesting features based on how the model was trained.  This does imply that the model was trained on images with some commonality to the new problem.

`So if your new dataset differs a lot from the dataset on which the original model was trained, you may be better off using on the first few layers of the model to do feature extraction, rather than using the entire convolutional base`.

The representation learned by the fully connected network layer will be specific to the new dataset that the model is trained on, as it will only contain information about the set of possible outcomes.





![CNN Arch](notebook_images/A-convolutional-neural-networks-CNN.png)

<br>
<br>

<span><font size="2">
    A New Method for Face Recognition Using Convolutional Neural Network - Scientific Figure on ResearchGate. Available from: https://www.researchgate.net/figure/A-convolutional-neural-networks-CNN_fig6_321286547 [accessed 18 Mar, 2020]
    </font></span>

This means we can use a trained model leveraging all of the time and data it took for training the convolutional part, and just remove the FCN layer.  

A properly trained CNN requires a lot of data and CPU/GPU time.  If our dataset is small, we cannot start from a fresh CNN model and try to train it and hope for good results. 

![cnnlayer](notebook_images/ccn_layer.png)

Leveraging the frozen CNN layer, we just have to train a new fully connected layer.  This requires much less CPU and while we would like as much data as is possible, we can obtain very good results with less data on the fully connected layer.

![cnnlayer](notebook_images/cnn_new_fcn.png)

## Applying Deep Transfer Learning

We will be using Tensorflow and Keras to build out a model for the COVID-19 Chest X-Ray detection.  

Keras comes with a number of models with their pre-configured architecture.  You can find out more about the Keras models [here](https://keras.io/applications/).  Pre-configured means that the exact architecture in terms of the number of layers and number of kernels is already configured.  But these models have no weights.  It is the training process that determines the weights to use, however, we can use the weights that have already be determined to work well using large amounts of image data.  For that we will use the imagenet weights for each of the models.

[ImageNet](http://imagenet.stanford.edu) is an image database of millions of images.  These images were used to train the model architectures and those best model weights can be made available so that we do not have to train a model from nothing.



We are going to look at 4 Keras models:

* VGG16 This was also the model that Adrian used in his [blog post](https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/).  More information can be found [here](https://arxiv.org/abs/1409.1556)

* VGG19  More information can be found [here](https://arxiv.org/abs/1409.1556)

* ResNet50  More information can be found [here](https://arxiv.org/abs/1512.03385)

* ResNet50V2  More information can be found [here](https://arxiv.org/abs/1603.05027)

The exact details of the models can be found from the links above

## Implementation Details

In [1]:
from build_covid_dataset import create_covid_dataset

### COVID-19 Dataset

Go to github and clone the repository:

https://github.com/ieee8023/covid-chestxray-dataset


In [ ]:
covid_github_dir = '/Volumes/MacBackup/covid-chestxray-dataset'
covid_output_dir = './dataset/covid'